Três funções principais serão implementadas e avaliadas/comparadas o desempenho de forma similar ao que foi implementado no "Guided Project: Building Fast Queries on a CSV" (presente no curso "Algorithm Complexity"):

- 1) dado um id de uma mensagem no Reddit, retornar todas as informações sobre a mensagem; 
- 2) dado um limite inferior e superior da coluna "sentiment", retornar todas as mensagens com valores de sentimento entre os limites inferior e superior; 
- 3) dado um valor de parâmetro, retornar duas mensagens cuja soma do valor da coluna "score" é igual ao parâmetro. Retornar -1 se não existir. 

- Implementar testes com o pytest para fins de validação.

In [11]:
import csv 
import time
import random

%pip install pytest pytest-sugar

Note: you may need to restart the kernel to use updated packages.


In [18]:
def row_sentiment(row):
    return row[8]

class ClimateChange():
    def __init__(self, dataset):
        with open(dataset, encoding = 'utf-8') as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]

        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row

    def get_message_from_id(self, message_id):
        for row in self.rows:
            if(row[0] == message_id):
                return row
        return None
    
    def get_message_from_id_fast(self, message_id):
        if(message_id in self.id_to_row):
            return self.id_to_row[message_id]
        return None 

In [17]:
climateChange = ClimateChange('the_reddit_climate_change_dataset_comments.csv')

In [25]:
print(climateChange.header)
print(climateChange.rows[0])

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']
['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', '2']
